<a href="https://colab.research.google.com/github/minsaee/ai_chat_python/blob/master/407_kakao%EB%A9%94%EC%8B%9C%EC%A7%80_%EA%B8%B0%ED%9B%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/ai_chat_python'

Mounted at /content/drive
/content/drive/MyDrive/ai_chat_python


In [ ]:
from bs4 import BeautifulSoup
import requests

# result dict으로 받기.
result = {}

# url 설정하기.
url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
res = requests.get(url)
print(res) # Response [200]

soup = BeautifulSoup(res.text, 'html.parser')

# 현재 시간 설정하기.
# id에서 value를 가져온다.
datetm = soup.select_one('#select-tm').get('value')
# print(datetm)
result['datetm'] = datetm

tbody_tag = soup.find('tbody')
tr_rs_tag = tbody_tag.find_all('tr')

for tr_tag in tr_rs_tag:
  td_rs_tag = tr_tag.find_all('td')

  city = td_rs_tag[0].string     # 도시이름
  if city == '서울':
    cur_temp = td_rs_tag[5].text # 현재기온
    sen_temp = td_rs_tag[7].string # 체감온도
    result['city']=city
    result['cur_temp']= cur_temp
    result['sen_temp']= sen_temp
    break

print(result)

import json
import datetime
import kakao_utils

# 토큰 가져오기.
KAKAO_TOKEN_FILENAME = './res/kakao_message/kakao_token.json'

# REST API KEY 가져오기.
KAKAO_APP_KEY = 'REST API Key'

# 토큰 업데이트하기.
tokens = kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)

# 날씨 정보 만들기.
text = f"""\
\ 날짜 정보 ({result['datetm']})
도시 : {result['city']}
현재온도 : {result['cur_temp']}
체감온도 {result['sen_temp']}
"""

# 텍스트 템플릿 형식 만들기
weather_url="https://www.weather.go.kr/w/obs-climate/land/city-obs.do?tm=2023.12.6.12:00&type=t99&mode=0&reg=100&auto_man=m&stn=108"
template = {
        'object_type' : 'text',
        'text': text,
        'link':{
            'web_url':weather_url,
            'mobile_web_url':weather_url
        },
        "button_title":"날씨 상세보기"
}


# 나에게 카카오톡 메시지 보내기 요청 (text type)
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)

if res.json().get('result_code') == 0:
  print('날씨 및 미세먼지 정보를 성공적으로 보냈습니다.')
else:
  print('날씨 및 미세먼지 정보를 보내지못했습니다. 오류메시지:', res.json())